### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 28 мая 2022, 08:30   
**Штраф за опоздание:** по 1 баллу за 24 часа задержки.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0221, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier


In [1102]:
import random
import math


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1
    
    def __init__(self, min_samples_split=2, max_depth=5, criterion='clf_err'):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
            'clf_err' -- Ошибка классификации,
            'gini'    -- Индекс Джини,
            'entropy' -- Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_class = -1
        # Для последнего задания
        self.feature_importances_ = None
        self.criterion = criterion
        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева 
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
        self.tree = dict()
        self.gain = np.zeros(1)
        
        self.summ = 0.0

    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    def __find_threshold_clf_err(self, x, y):
        best_feature_id = -1
        best_thr = 0
        best_imp = 0
        temp_best_0 = 0
        temp_best_1 = 0
        R_left_stat = np.zeros(self.num_class, dtype=int)
        R_right_stat = np.zeros(self.num_class, dtype=int)
        R = len(y)
        
        for feature_id in range(x.shape[1]):
            calced = x[:, feature_id]
            coords = calced.argsort()
            calced = calced[coords]
            y = y[coords]
            
            some_stat = np.array(y)
            some_stat[:-1] -= y[1:]
            some_stat[-1] = 1
            mask = some_stat != 0
            stat_y = y[mask]
            stat_count = np.arange(R)[mask]
            stat_count[1:] -= np.array(stat_count[:-1])
            stat_count[0] += 1

            R_left = 0
            R_right = R
            R_left_stat.fill(0)
            R_right_stat.fill(0)

            for cur_y, cur_count in zip(stat_y, stat_count):
                R_right_stat[cur_y] += cur_count
                
            R_max = R_right_stat.max()
            
            H_R = R_max * R
            R_left_max = 0
            R_right_max = R_max
            
            temp_best_1 = R * R_max
            
            for ind, cls, amount in zip(range(R), stat_y, stat_count):
                R_left += amount
                R_right -= amount
                left_save = R_left_stat[cls] + amount
                R_left_stat[cls] = left_save
                right_save = R_right_stat[cls]
                R_right_stat[cls] = right_save - amount
                
                if left_save > R_left_max:
                    R_left_max = left_save
                if right_save == R_right_max:
                    R_right_max = R_right_stat.max()
                
                cur_impurity = R_left_max * R_left + R_right_max * R_right
                
                if cur_impurity < temp_best_1:
                    temp_best_0 = ind
                    temp_best_1 = cur_impurity
            
            if best_feature_id == -1 or temp_best_1 < best_imp:
                best_feature_id = feature_id
                best_thr = 0
                for ind, predicat in enumerate(mask):
                    if predicat:
                        if best_thr == temp_best_0:
                            best_thr = ind
                            break
                        best_thr += 1               
                best_thr = (calced[best_thr] + calced[best_thr + 1]) / 2
                best_imp = temp_best_1
        
        self.gain[best_feature_id] += best_imp / (R ** 2)
        
        return best_feature_id, best_thr

    def __find_threshold_gini(self, x, y):
        best_feature_id = -1
        best_thr = 0
        best_imp = 0
        temp_best_0 = 0
        temp_best_1 = 0
        R_left_stat = np.zeros(self.num_class, dtype=int)
        R_right_stat = np.zeros(self.num_class, dtype=int)
        R = len(y)
        
        for feature_id in range(x.shape[1]):
            calced = x[:, feature_id]
            coords = calced.argsort()
            calced = calced[coords]
            y = y[coords]
            
            some_stat = np.array(y)
            some_stat[:-1] -= y[1:]
            some_stat[-1] = 1
            mask = some_stat != 0
            stat_y = y[mask]
            stat_count = np.arange(R)[mask]
            stat_count[1:] -= np.array(stat_count[:-1])
            stat_count[0] += 1

            R_left = 0
            R_right = R
            R_left_stat.fill(0)
            R_right_stat.fill(0)

            for cur_y, cur_count in zip(stat_y, stat_count):
                R_right_stat[cur_y] += cur_count
            
            R_sum = (R_right_stat ** 2).sum()
            R_right_sum = R_sum
            R_left_sum = 0
            H_R = R_sum * R
            
            temp_best_1 = H_R * R
            
            for ind, cls, amount in zip(range(R), stat_y, stat_count):
                R_left += amount
                R_right -= amount
                left_save = R_left_stat[cls]
                R_left_stat[cls] = left_save + amount
                right_save = R_right_stat[cls]
                R_right_stat[cls] = right_save - amount
                
                sq = amount ** 2
                R_left_sum += sq + 2 * amount * left_save
                R_right_stat += sq - 2 * amount * right_save
                
                cur_impurity = R_left * R_left_sum + R_right * R_right_sum
                
                if cur_impurity < temp_best_1:
                    temp_best_0 = ind
                    temp_best_1 = cur_impurity
            
            if best_feature_id == -1 or temp_best_1 < best_imp:
                best_feature_id = feature_id
                best_thr = 0
                for ind, predicat in enumerate(mask):
                    if predicat:
                        if best_thr == temp_best_0:
                            best_thr = ind
                            break
                        best_thr += 1               
                best_thr = (calced[best_thr] + calced[best_thr + 1]) / 2
                best_imp = temp_best_1
        
        self.gain[best_feature_id] += best_imp / (R ** 3)
        
        return best_feature_id, best_thr

    def __find_threshold_entropy(self, x, y):
        best_feature_id = -1
        best_thr = 0
        best_imp = 0
        temp_best_0 = 0
        temp_best_1 = 0
        R_left_stat = np.zeros(self.num_class, dtype=int)
        R_right_stat = np.zeros(self.num_class, dtype=int)
        R = len(y)
        
        for feature_id in range(x.shape[1]):
            calced = x[:, feature_id]
            coords = calced.argsort()
            calced = calced[coords]
            y = y[coords]
            
            some_stat = np.array(y)
            some_stat[:-1] -= y[1:]
            some_stat[-1] = 1
            mask = some_stat != 0
            stat_y = y[mask]
            stat_count = np.arange(R)[mask]
            stat_count[1:] -= np.array(stat_count[:-1])
            stat_count[0] += 1

            R_left = 0
            R_right = R
            R_left_stat.fill(0)
            R_right_stat.fill(0)

            for cur_y, cur_count in zip(stat_y, stat_count):
                R_right_stat[cur_y] += cur_count
            
            temp = R_right_stat / R
            temp[temp == 0] = 1
            R_sum = -1 * (temp * np.log(temp)).sum()
            R_left_sum = 0
            R_right_sum = R_sum
            
            temp_best_1 = R * R_sum * R
            
            for ind, cls, amount in zip(range(len(stat_y) - 1), stat_y, stat_count):
                R_left += amount
                R_right -= amount
                left_save = R_left_stat[cls]
                R_left_stat[cls] = left_save + amount
                right_save = R_right_stat[cls]
                R_right_stat[cls] = right_save - amount
                
                if left_save != 0:
                    R_left_sum += math.log(left_save / R) * left_save / R
                left_save += amount
                R_left_sum -= math.log(left_save / R) * left_save / R
                if right_save != 0:
                    R_right_sum -= math.log(right_save / R) * right_save / R
                right_save += amount
                R_right_sum += math.log(right_save / R) * right_save / R
                
                cur_impurity = R_left * R_left_sum + R_right * R_right_sum
                
                if cur_impurity < temp_best_1:
                    temp_best_0 = ind
                    temp_best_1 = cur_impurity
            
            if best_feature_id == -1 or temp_best_1 < best_imp:
                best_feature_id = feature_id
                best_thr = 0
                for ind, predicat in enumerate(mask):
                    if predicat:
                        if best_thr == temp_best_0:
                            best_thr = ind
                            break
                        best_thr += 1               
                best_thr = (calced[best_thr] + calced[best_thr + 1]) / 2
                best_imp = temp_best_1
        
        self.gain[best_feature_id] += R_sum + best_imp
        
        return best_feature_id, best_thr
    
    def __find_threshold(self, x, y):
        """
        Находим оптимальный признак и порог для сплита
        Здесь используем разные impurity в зависимости от self.criterion
        """
        if self.criterion == 'clf_err':
            return self.__find_threshold_clf_err(x, y)
        elif self.criterion == 'gini':
            return self.__find_threshold_gini(x, y)
        elif self.criterion == 'entropy':
            return self.__find_threshold_entropy(x, y)
        else:
            raise ValueError()
        

    def __fit_node(self, x, y, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с айди 2 * node_id + 2
        """
        # Ваш код здесь
        
        # empty node
        if x.shape[0] == 0:
            self.tree[node_id] = (self.__class__.LEAF_TYPE, random.randint(0, self.num_class), 1 / self.num_class)
            return
                
        # uniform, min saples, max depth
        stat_y, stat_counts = np.unique(y, return_counts=True)
        arg = stat_counts.argmax()
        max_counts = stat_counts[arg]
        arg = stat_y[arg]
        
        if x.shape[0] == max_counts or x.shape[0] <= self.min_samples_split or depth >= self.max_depth:
            self.tree[node_id] = (self.__class__.LEAF_TYPE, arg, max_counts / len(y))
            return
        
        # must do split
        feature_id, threshold = self.__find_threshold(x, y)
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, feature_id, threshold)
        
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, feature_id, threshold)
        self.__fit_node(x_left, y_left, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_right, y_right, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.num_class = np.unique(y).size
        self.gain = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]
        
    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    
    def get_feature_importance(self):
        """
        Возвращает важность признаков
        """
        return self.gain

In [1093]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [1094]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)
my_clf.fit(X_train, y_train)
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [1095]:
print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))
print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.9444444444444444
0.8888888888888888


## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [1097]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [1098]:
%time clf.fit(X_train, y_train)

Wall time: 1.31 ms


DecisionTreeClassifier()

In [1099]:
%time my_clf.fit(X_train, y_train)

Wall time: 25 ms


## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности. 
Постройте графики зависимости точности на валидации от глубины дерева, от минимального числа объектов для сплита. 
Какой максимальной точности удалось достигнуть?

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

In [1105]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion='entropy')
my_clf.fit(X_train, y_train)

In [1106]:
my_clf.get_feature_importance()

array([255.78834867,  59.1716321 ,   6.17485689,  28.55175613,
         0.        ,   0.        ,   8.2311727 ,  46.44879485,
        24.3841442 ,  33.53171272,  20.25240445,  10.77768938,
         0.        ])

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

